In [2]:
import numpy as np

class Matrix:
    __array_priority__ = 1
    
    def __init__(self, n, m, np_matrix):
        self.n = n
        self.m = m
        self.np_matrix = np_matrix.copy()
        
        self.matrix = [(i, j, np_matrix[i, j])
                      for i in range(n)
                      for j in range(m)
                      if np_matrix[i, j] != 0]
        
    def transpose(self):
        res = np.zeros((self.m, self.n))
        for i, j, elem in self.matrix:
            res[j, i] = elem
        return Matrix(self.m , self.n, res) 
    
    def __matmul__(self, other):
        if self.m != other.shape[0]:
            raise Exception("Vector's length differs from number of columns")
            
        prod = np.zeros(self.n)
        for i, j, elem in self.matrix:
            prod[i] += elem * other[j]
        return prod
        
    def __rmatmul__(self, other):
        
        if self.n != other.shape[0]:
            raise Exception("Vector's len differs from number of rows")

        prod = np.zeros(self.m)
        for i, j, elem in self.matrix:
            prod[j] += elem * other[i]
        return prod
        
        

In [3]:
for i in range(1000):
    n, m = np.random.randint(1, 100, 2)

    arr = np.random.rand(n, m)

    Matr = Matrix(n, m, arr)

    a = np.random.rand(m)
    b = np.random.rand(n)

    rmul = arr @ a

    lmul = b @ arr

    rmul1 = Matr @ a
    lmul1 = b @ Matr

    assert(np.all(abs(rmul - rmul1) < 1e-4))
    assert(np.all(abs(lmul - lmul1) < 1e-4))
    assert(np.all(abs(Matr.transpose().np_matrix - arr.T)) < 1e-4)



$$\| Ax - b \| = \frac{1}{2}\|x^T(2A^TA)x\| + \|(2A^Tb)^Tx\| + Const$$

Тогда просто применяем метод при $A = A^TA$, $b = A^Tb$

In [4]:
def conjugate_grad(A, b):
    x = np.zeros(A.m)
    AT = A.transpose()
    b = AT @ b
    v = AT @ (A @ x) - b
    d = v
    v_norm = np.dot(v, v)
    
    for i in range(len(b)):
        Ad = AT @ (A @ d)
        alpha = v_norm / (d @ Ad)
        x = x - alpha * d
        v = v - alpha * Ad
        v_norm_new = np.dot(v, v)
        d = v + (v_norm_new / v_norm) * d
        
        v_norm = v_norm_new
    
    return x
    

__TEST__:

In [17]:
import cvxpy as cp

def cv_sol(A, b):
    x = cp.Variable(A.shape[1])
    obj = cp.Minimize(cp.sum_squares(A @ x - b))
    con = []
    cp.Problem(obj, con).solve()
    return x.value

n, m = 5, 5

for i in range(100):
    A = np.random.rand(n, m)
    b = np.random.rand(n)


    res = conjugate_grad(Matrix(n, m, A), b)
    cv_res = cv_sol(A, b)
    
    for i in range(m):
        assert(abs(res[i] - cv_res[i]) < 1e-3)